In [2]:
# Esperimenti sull'output di inferenza a mano
import tensorflow as tf
import numpy as np
from PIL import Image

nomeModello = "Modelli/Utili/yolo11n_float32.tflite"
nomeImmagine = "Foto/example4_preprocessed.jpg"

#converte l'immagine in RGB
img = Image.open(nomeImmagine).convert('RGB')

# Converti in un tensore 3d con valori float32 e normalizza (valori da 0 a 1)
input_data =  np.array(img).astype(np.float32) / 255.0

#aggiungi dimensione batch (1, 640, 640, 3) (Yolo si aspetta sempre un batch di input)
#YOLO richiede input di forma (batch_size, height, width, channels) (ottenendo un tensore 4d)
input_data = np.expand_dims(input_data, axis=0)

#carica il modello
interpreter = tf.lite.Interpreter(model_path=nomeModello)
interpreter.allocate_tensors() #alloca memoria per i tensori

#formati input/output modello
input_details = interpreter.get_input_details()
output_details = interpreter.get_output_details()
print("formato dell'input del modello:", input_details[0]['dtype'])
print("formato output del modello:", output_details[0]['dtype'])

# --- Imposta tensore input ---
interpreter.set_tensor(input_details[0]['index'], input_data) 

# --- Esegui inferenza ---
interpreter.invoke()

# --- Ottieni output ---
output_data = interpreter.get_tensor(output_details[0]['index'])
predictions = output_data[0]

# --- Analisi output ---
print("Shape output:", output_data.shape)
predictions = predictions.T #faccio la trasposta per avere su ogni riga una predizione
print("predictions shape:", predictions.shape)
#print("predictions:", predictions)
print("prima riga:", predictions[1002])

objectness = predictions[:, 4]
class_0_probs = predictions[:, 5]







formato dell'input del modello: <class 'numpy.float32'>
formato output del modello: <class 'numpy.float32'>
Shape output: (1, 84, 8400)
predictions shape: (8400, 84)
prima riga: [5.40647328e-01 1.51085123e-01 8.51498842e-02 4.00711894e-02
 2.79087430e-07 6.26703880e-08 3.29604177e-07 6.01762480e-08
 1.32663345e-07 7.62738921e-08 7.28267437e-08 6.29471231e-08
 1.61372540e-07 6.91916711e-08 1.55344235e-08 6.17176070e-08
 2.56964405e-08 1.74080597e-07 2.06534068e-07 6.85886832e-08
 5.10500868e-08 7.56553575e-08 5.57399922e-08 8.17181132e-08
 1.03012141e-07 6.16727718e-08 7.68923556e-08 7.43975193e-08
 1.57854444e-07 3.22302526e-07 1.44844464e-07 1.74556220e-07
 7.69884778e-08 1.58081775e-07 2.65187907e-07 9.33271380e-08
 7.09081931e-08 1.48563984e-07 2.10989526e-07 4.97726766e-08
 6.40138040e-08 2.40848721e-07 8.98006363e-08 4.40786572e-08
 3.20233156e-08 6.13186657e-08 2.64282818e-07 2.86992559e-07
 2.62695181e-07 1.63428197e-07 2.01161612e-07 1.06888720e-07
 4.81617128e-08 8.73457680e-0

In [50]:
#Usiamo la libreria ultralytics per fare inferenza con YOLO11n
from ultralytics import YOLO

# Load a model
model = YOLO("yolov8n.pt") #esporta in .pt

In [ ]:
#verifica dimensione input del modello
import onnx

model = onnx.load("yolov8n.onnx")
for input in model.graph.input:
    print(input.name)
    shape = [dim.dim_value for dim in input.type.tensor_type.shape.dim]
    print("Input shape:", shape)


images
Input shape: [1, 3, 640, 640]


In [ ]:
model.export(format="onnx") #esporta in .onnx

In [ ]:
# Inference and save results with Ultralytics
nomeImmagine = "../../Calibrazione/immagini_calibrazione640x480/capture(1).jpg"
results = model.predict(
    nomeImmagine, 
    save=True, 
    project="../../FotoInference",
    name="detection",
)
print("results:", results)

In [ ]:
#quantizzazione del modello che non funziona
import sys, os
import esp_ppq
#sys.path.append(os.path.abspath("esp-detection"))
from deploy.quantize import quant_espdet

import onnx
from onnxsim import simplify

quant_espdet(
    onnx_path="yolo11n.onnx",      # onnx appena esportato
    target="esp32s3",              
    num_of_bits=8,                 # quantizzazione int8
    device='cpu',
    batchsz=1,
    imgsz=[640, 640],              # deve combaciare con la dimensione del modello
    calib_dir="../../Calibrazione/immagini_ridimensionate640x640/",        #  cartella immagini reali
    espdl_model_path="yolo11n.espdl"
)
